## Fetching and Saving Data from Open Charge Map API


In [1]:
import requests
import json
import datetime

# URL of the API endpoint
url = "https://api.openchargemap.io/v3/poi/"

# Query parameters (adjust these as needed)
querystring = {
    "output": "json",
    "countrycode": "GB",  
    "key": '',  # API key
    "maxresults": 200000,
}

# Make the GET request
response = requests.get(url, params=querystring)

# Check for successful response
if response.status_code == 200:
    # Parse JSON response
    data = response.json()

    # Get the current date and time
    current_time = datetime.datetime.now()
    
    # Format the date and time as a string in the format "YYYY-MM-DD_HH-MM-SS"
    time_string = current_time.strftime("%Y-%m-%d_%H-%M-%S")

    # Define the output file path, including the date and time
    country_code = querystring['countrycode']
    output_file_path = f"output_data_{country_code}_{time_string}.json"

    # Save the data to the specified file
    with open(output_file_path, 'w') as file:
        json.dump(data, file, indent=4)

    print(f"Data saved to {output_file_path}")
else:
    print("Failed to retrieve data: ", response.status_code)

Data saved to output_data_GB_2024-05-31_14-52-48.json


## Identifying Common Values
Identifying common values by comparing each dictionary within the duplicate_pairs dataset.

In [2]:
import pandas as pd

# Load the JSON data from the file
with open(r"output_data_GB_2024-05-31_14-00-11.json", 'r') as json_file:
    data = json.load(json_file)  # Assuming data is a list of dictionaries

# Function to flatten a nested dictionary and handle lists with nested dictionaries
def flatten_dict(dd, separator='_', prefix=''):
    flattened = {}
    for k, v in dd.items():
        new_key = prefix + separator + k if prefix else k
        if isinstance(v, dict):
            flattened.update(flatten_dict(v, separator, new_key))
        elif isinstance(v, list):
            # Flatten each dictionary in the list and then convert the list to a string
            flattened_list = [flatten_dict(item, separator, new_key) if isinstance(item, dict) else item for item in v]
            flattened[new_key] = str(flattened_list)
        else:
            flattened[new_key] = v
    return flattened

# Flatten all dictionaries in the dataset
flattened_data = [flatten_dict(item) for item in data]

# Function to find duplicates across all dictionaries, excluding None values
def find_duplicates(data):
    value_counts = {}
    for item in data:
        for key, value in item.items():
            if value is not None:  # Skip None values
                value_counts.setdefault((key, value), 0)
                value_counts[(key, value)] += 1
    return {k: v for k, v in value_counts.items() if v > 1}

# Find duplicates
duplicates = find_duplicates(flattened_data)

# Convert duplicates dictionary to a DataFrame
duplicates_df = pd.DataFrame([(k[0], k[1], v) for k, v in duplicates.items()], columns=['Key', 'Value', 'Count'])

# Display the DataFrame
duplicates_df.sort_values(by=['Count'], ascending=False).head(40)

# Analyze and print the results
#print(f"Total number of duplicate entries (excluding None values): {len(duplicates)}")
#for k, v in duplicates.items():
#    print(f"Duplicate Key-Value: {k}, Count: {v}")

# Save the DataFrame to an Excel file
excel_file_path = r"duplicates_data.xlsx"
duplicates_df.to_excel(excel_file_path, index=False)

print(f"DataFrame saved to {excel_file_path}")
duplicates_df


DataFrame saved to duplicates_data.xlsx


,Key,Value,Count
0,DataProvider_WebsiteURL,http://openchargemap.org,15501
1,DataProvider_DataProviderStatusType_IsProvider...,True,27295
2,DataProvider_DataProviderStatusType_ID,1,15501
3,DataProvider_DataProviderStatusType_Title,Manual Data Entry,15501
4,DataProvider_IsRestrictedEdit,False,27295
...,...,...,...
17469,AddressInfo_Latitude,51.510944,2
17470,AddressInfo_Longitude,-0.146848,2
17471,DateLastConfirmed,2011-04-15T00:00:00Z,10
17472,DateCreated,2011-04-15T00:00:00Z,10


## Analyzing Postcodes in JSON Data


In [3]:
# Load the JSON data from the input file
input_file_path = r"output_data_GB_2024-05-31_14-00-11.json"

# Initialize counters
numerical_postcode_count = 0
zip_plus_4_count = 0
text_postcode_count = 0
other_postcode_count = 0

# Create lists to store different types of postcodes and their examples
text_postcodes = []
other_postcodes = []
numerical_postcode_examples = []
zip_plus_4_examples = []
text_postcode_examples = []
other_postcode_examples = []

# Load the JSON data from the input file
with open(input_file_path, 'r') as json_file:
    data = json.load(json_file)

# Iterate through the data and check the "Postcode" field
for item in data:
    address_info = item.get("AddressInfo", {})
    postcode = address_info.get("Postcode")

    if postcode is not None:
        if postcode.isdigit():
            numerical_postcode_count += 1
            if len(numerical_postcode_examples) < 10:  # Collect up to 10 examples
                numerical_postcode_examples.append(postcode)
        elif len(postcode.split('-')) == 2 and all(part.isdigit() for part in postcode.split('-')):
            zip_plus_4_count += 1
            if len(zip_plus_4_examples) < 10:
                zip_plus_4_examples.append(postcode)
        elif any(char.isalpha() for char in postcode):
            text_postcode_count += 1
            text_postcodes.append(postcode)
            if len(text_postcode_examples) < 10:
                text_postcode_examples.append(postcode)
        else:
            other_postcode_count += 1
            other_postcodes.append(postcode)
            if len(other_postcode_examples) < 10:
                other_postcode_examples.append(postcode)

# Print the results
print(f"Number of numerical postcodes: {numerical_postcode_count}")
print(f"Examples: {numerical_postcode_examples}")
print(f"Number of ZIP+4 postcodes (US): {zip_plus_4_count}")
print(f"Examples: {zip_plus_4_examples}")
print(f"Number of text postcodes: {text_postcode_count}")
print(f"Examples: {text_postcode_examples}")
print(f"Number of other postcodes: {other_postcode_count}")
print(f"Examples: {other_postcode_examples}")

Number of numerical postcodes: 2
Examples: ['25240', '27030']
Number of ZIP+4 postcodes (US): 0
Examples: []
Number of text postcodes: 27238
Examples: ['EX31 3RZ', 'WD6 5NA', 'PL27 6QJ', 'OX7 5SY', 'BD1 4RB', 'B80 7DR', 'DN17 1BL', 'B1 1NQ', 'S26 7XR', 'CM15 8AY']
Number of other postcodes: 25
Examples: ['152 564', '', '', '1.544678', '', '', '', '', '', '']


## Loading and Flattening JSON Data


In [7]:
# Load the JSON data into a list of dictionaries
with open(r"output_data_GB_2024-05-31_14-00-11.json", 'r') as json_file:
    data = json.load(json_file)

# Normalize the Data (flatten the nested dictionaries)
normalized_df = pd.json_normalize(data)

# Create a DataFrame 
df=pd.DataFrame(data)

# Convert the list of dictionaries into a DataFrame
df_norm = pd.DataFrame(data=normalized_df)

# Print summary statistics of the DataFrame
df_norm.head()

,UserComments,PercentageSimilarity,MediaItems,IsRecentlyVerified,DateLastVerified,ID,UUID,ParentChargePointID,DataProviderID,DataProvidersReference,...,AddressInfo.ContactTelephone1,AddressInfo.ContactTelephone2,AddressInfo.ContactEmail,AddressInfo.AccessComments,AddressInfo.RelatedURL,AddressInfo.Distance,AddressInfo.DistanceUnit,OperatorInfo,StatusType,UsageType
0,None,None,None,True,2024-05-30T13:25:00Z,299633,F2B8ED7F-B0ED-488D-B4E3-FC3975EE1D6A,None,1,None,...,None,None,None,None,None,None,0,NaN,NaN,NaN
1,None,None,None,True,2024-05-30T05:17:00Z,299596,56911CBC-0E1A-4D0B-9C89-844D255CD53B,None,1,None,...,None,None,None,None,None,None,0,NaN,NaN,NaN
2,None,None,None,True,2024-05-30T05:17:00Z,299595,1C7B554F-4B3A-45DF-9B8C-FE23FEAC327B,None,1,None,...,None,None,None,None,None,None,0,NaN,NaN,NaN
3,None,None,"[{'ID': 42456, 'ChargePointID': 299585, 'ItemU...",True,2024-05-31T08:03:00Z,299585,310FE1F0-E7E8-4045-840F-5AE1A650B67B,None,1,None,...,None,None,None,None,None,None,0,NaN,NaN,NaN
4,None,None,None,True,2024-05-29T13:09:00Z,299584,0C449EB3-9A23-4C1A-9A76-498C406A81D2,None,1,None,...,None,None,None,None,None,None,0,NaN,NaN,NaN


## Displaying Summary Statistics of the DataFrame


In [15]:
# Get the count of non-null values for each column
total = df.count()

# Calculate the number of missing values compared to the first column
missing_data = total.iloc[0] - total

# Calculate the percentage of missing values compared to the first column
percentage_missing = ((missing_data / total.iloc[0]) * 100).round(2)

# Adding these as new columns in a DataFrame
total = pd.DataFrame(total, columns=['Count'])
total['Missing'] = missing_data
total['% Missing'] = percentage_missing

# Print the DataFrame
total.style.highlight_min(subset=["Missing", "% Missing"])


,Count,Missing,% Missing
DataProvider,27295,0,0.000000
OperatorInfo,27077,218,0.800000
UsageType,27252,43,0.160000
StatusType,16969,10326,37.830000
SubmissionStatus,27295,0,0.000000
UserComments,1685,25610,93.830000
PercentageSimilarity,0,27295,100.000000
MediaItems,1263,26032,95.370000
IsRecentlyVerified,27295,0,0.000000
DateLastVerified,27295,0,0.000000
